# 使用 Nsight Compute 优化在线 5G 机器学习

## 05 每个块多个样本

在此步骤中，线程组已被划分为图块，因此每个块可以处理多个样本。一旦使用协作组实现内核（本教程的起始版本已经是这种情况），这很容易做到。这里的目标是最大化内核对 GPU 的占用率，目标是 100% 的理论占用率。

优化已在代码中准备好。要启用新版本，我们只需在 [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h) 第 73 行设置 `APSM_DETECT_VERSION` 标志。将此定义设置为 `apsm_version::APSM_DETECT_SPB` 后，使用以下命令重新编译代码：

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!make -j

打开 [apsm_detect.cu](apsm/cpp/lib/apsm/apsm_detect.cu) 的第 400 行以检查内核之间的差异。

我们可以像以前一样以相同的交互方式对优化的内核进行分析，但这次我们也可以尝试使用 Nsight Compute 的 `ncu` 命令行界面 (CLI)，特别是因为我们已经确切知道要分析哪个内核。以下命令使用 CLI 指定内核的名称、要收集的部分/指标集以及输出报告文件的名称。我们也可以直接在控制台上检查结果，但在 UI 中将其与以前的版本进行比较更容易。

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!ncu -k kernel_apsm_detect --set full -f -o /dli/task/ncu/report_spb bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

单击下方 `...` 以查看注释。

收集结果后，切换到带有 Nsight Compute 的 Ubuntu 实例并打开刚刚创建的报告文件 `/root/Desktop/reports/ncu/report_spb.ncu-rep` 。打开后，返回第一个 CG 报告（如果您未保存，则可能名为“Untitled”）并激活 `Add Baseline` 以将旧结果添加为比较的基线。然后，再次切换到新的 SPB 报告，该报告现在将所有当前结果与未优化的基线数据进行了比较。

<img src="images/ncu_report02_01.png">

需要注意的主要一点是，总体性能（无论是在运行时还是在计算或内存利用率方面）并没有真正改变。这很不幸，我们需要找出原因。但是，为了验证更改在 `occupancy` 方面的表现是否符合预期，让我们先看看该部分。

<img src="images/ncu_report02_02.png">

正如我们计划的那样，占用率有所改善，理论占用率现在达到 GPU 最大值的 100%。实际占用率和活动扭曲的比例相同。由于改善占用率是改善每个调度程序理论扭曲的一种手段，因此我们也应该检查该部分。

<img src="images/ncu_report02_03.png">

在“调度程序统计”部分，性能随着理论值和实际值的新占用率而改善。此外，“每个调度程序的合格扭曲”也提高了 20%。不幸的是，它并没有为“每个调度程序发出的扭曲”带来明显的性能提升（理想情况下应该接近 1）。

那么为什么发出的扭曲没有改善呢？两个版本整体性能保持不变的原因可以在 `Warp State Statistics` 部分中找到：指令导致的停顿增加了 90%，这主要是由于 LG Throttle 停顿的增加。这种停顿类型表示等待 L1 指令队列进行本地和全局 (LG) 内存操作的 Warp 未满。通常，只有在非常频繁地执行本地或全局内存指令时才会发生这种情况。

<img src="images/ncu_report02_04.png">

Warp States 图表仅向我们显示了内核运行时平均的停顿原因。要更详细地确定它们在代码中发生的位置，我们可以查看“详细信息”页面末尾的“源计数器”部分：

<img src="images/ncu_report02_05.png">

除了一些关于代码分支行为的统计数据外，还有三个表格分别显示了停顿原因和执行指令的前 5 个源位置。由于每个指令的 Warp 状态是使用 PC 采样收集的，因此这里将它们称为“采样数据”。两个图表“采样数据（全部）”和“采样数据（未发出）”突出显示了此类 Warp 状态的两个方面：

* 每当采样的调度程序出现 Warp 停滞时，“全部”数据在收集过程中就会增加。
* 当采样的调度程序出现 Warp 停滞且同一调度程序槽中没有其他 Warp 可以发出时，“未发出”数据在收集过程中就会增加。

我们可以说，“采样数据（全部）”不能很好地解释延迟隐藏，但可以很好地显示总共有多少 Warp 停滞，而“采样数据（未发出）”则特别突出显示了无法通过发出另一个 Warp 来隐藏停滞延迟的位置。

尽管对于这个特定的内核，两者之间没有太大区别，但我们将重点关注“未发出”指标，因为这里的高值会对您的性能造成更大的问题。如果您点击具有最高度量值的源位置的链接（约占所有样本的 11%），您将进入“源”页面，并选择匹配的代码位置。

<img src="images/ncu_report02_06.png">

您会注意到，该页面当前仅显示内核汇编 (SASS) 指令。这是因为我们没有使用行信息构建内核，即编译器没有将源代码和汇编之间的任何关联信息嵌入到二进制文件中。

让我们修复构建脚本中缺少的行信息，以便我们可以拥有正确的 CUDA-C/SASS 关联。我们只需要在 [CMakeLists.txt](apsm/cpp/CMakeLists.txt) 的第 97-99 行启用 `-lineinfo`

```cmake
96 # Lineinfo (--generate-line-info/-lineinfo)
97 #if ( NOT "${CMAKE_CUDA_FLAGS}" MATCHES "-lineinfo" )
98 # list ( APPEND CUDA_NVCC_FLAGS "-lineinfo" )
99 #endif ()
```

启用后，重新构建应用程序并重新收集报告。请注意，从现在开始我们也使用 `--import-source yes`。虽然只要报告是在编译代码的同一系统上打开的（这样工具就可以找到二进制文件中嵌入的文件路径），这并不是绝对必要的，但如果想要移动报告或保留带有配置文件结果的原始源，这在许多情况下仍然很有用。

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!cmake ..
!make -j
!ncu -k kernel_apsm_detect --set full --import-source yes -f -o /dli/task/ncu/report_spb_lineinfo bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

打开新报告时，我们会在“来源”页面上看到类似的性能数据，但现在我们可以正确地将其与高级代码关联起来：

<img src="images/ncu_report02_07.png">

在[步骤 06](06_shmem.ipynb)中继续进行优化